In [3]:
import mediapipe as mp
import numpy as np
import cv2
import uuid
import os


In [4]:
mp_drawing=mp.solutions.drawing_utils
mpPose=mp.solutions.pose
pose=mpPose.Pose()
mp_hands=mp.solutions.hands
joint_list=[[4,3,2],[8,7,6],[12,11,10],[16,15,14],[20,19,18]]

In [5]:
def getLabel(index,hand,results):
    output=None
    for idx,classification in enumerate(results.multi_handedness):
        
        if classification.classification[0].index==index:
            label=classification.classification[0].label
            score=classification.classification[0].score
            text='{} {}'.format(label,round(score,2))
            
            coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                [640,480]).astype(int)
            )
            output=text,coords
    return output
        

In [6]:
def draw_angles(image,results,joint_list):
    for hand in results.multi_hand_landmarks:
        for joint in joint_list:
            a=np.array([hand.landmark[joint[0]].x,hand.landmark[joint[0]].y])
            b=np.array([hand.landmark[joint[1]].x,hand.landmark[joint[1]].y])
            c=np.array([hand.landmark[joint[2]].x,hand.landmark[joint[2]].y])
            radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
            angle=np.abs(radians*180.0/np.pi)
            
            if angle>180.0:
                angle=360-angle
            cv2.putText(image,str(round(angle,2)),tuple(np.multiply(b,[640,480]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(190,190,190),2,cv2.LINE_AA)
    return image
            

In [8]:
cap=cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=4) as hands:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image=cv2.flip(image,1)
        image.flags.writeable=False
        results=hands.process(image) 
        #res=poses.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        #print(results.multi_hand_landmarks)
        if results.multi_hand_landmarks:
            for num,hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(180,180,180),thickness=2,circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(180,180,180),thickness=2,circle_radius=2),
                                         )
                if getLabel(num,hand,results):
                    text,coord=getLabel(num,hand,results)
                    cv2.putText(image,text,coord,cv2.FONT_HERSHEY_DUPLEX,1,(190,190,190),2,cv2.LINE_AA)
            draw_angles(image,results,joint_list)
        '''
        if res.pose_landmarks:
            for num,p in enumerate(res.pose_landmarks):
                mp_drawing.draw_landmarks(image,p,mpPose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(180,180,180),thickness=2,circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(180,180,180),thickness=2,circle_radius=2),
                                         )
        '''
        #cv2.imwrite(os.path.join('Hands Output','{}.jpg'.format(uuid.uuid1())),image)
        cv2.imshow('Tracking',image)  
        
    
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()